In [6]:
import googleapiclient.discovery
import pandas as pd

def get_youtube_service():
    api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)
    return youtube

def search_channels(youtube, query, page_token=None, max_results=50):
    request = youtube.search().list(
        part="snippet",
        q=query,
        type="channel",
        maxResults=max_results,
        pageToken=page_token
    )
    response = request.execute()
    return response

def get_channel_details(youtube, channel_ids):
    request = youtube.channels().list(
        part="snippet,statistics",
        id=",".join(channel_ids)
    )
    response = request.execute()
    return response

def main():
    youtube = get_youtube_service()
    
    global channels
    channels = []
    queries = ['data analytics', 'data analysis', 'data science']
    
    for query in queries:
        page_token = None
        while len(channels) < 500:
            # Search for channels related to 'data analytics'
            search_response = search_channels(youtube, query, page_token=page_token)

            channel_ids = [item['id']['channelId'] for item in search_response['items']]

            # Get details for these channels
            for i in range(0, len(channel_ids), 10):  # Process in batches of 10 to stay within quota limits
                batch_ids = channel_ids[i:i + 10]
                channel_details = get_channel_details(youtube, batch_ids)

                for channel in channel_details['items']:
                    view_count = int(channel['statistics']['viewCount'])
                    if view_count > 100000:
                        channels.append({
                            'title': channel['snippet']['title'],
                            'description': channel['snippet']['description'],
                            'subscriber_count': int(channel['statistics']['subscriberCount']),
                            'video_count': int(channel['statistics']['videoCount']),
                            'view_count': view_count,
                            'joined_date': channel['snippet']['publishedAt'],
                            'url': f"https://www.youtube.com/channel/{channel['id']}"
                        })
                        if len(channels) >= 500:
                            break
                if len(channels) >= 500:
                    break
        
            page_token = search_response.get('nextPageToken')
            if not page_token:
                break
    
def insert_into_dataframe(channels):
    
    #Add column names
    column_names = ['Title','Description','Subscriber Count','Video Count','View Count','Joined Date','URL']
    
    # Create DataFrame from channels list
    df = pd.DataFrame(channels)
    
    #Dropping the duplicates
    df.drop_duplicates(subset = 'url', keep = 'first', inplace = True)
    
    # Print the DataFrame
    print(df)
    
    # Optionally, save the DataFrame to a CSV file
    df.to_csv('D:/My learning/Projects/YouTube API Data Analysis/abc.csv', index=False)
    
    return df

if __name__ == "__main__":
    main()

    # Insert the channel data into a DataFrame
    df = insert_into_dataframe(channels)
    print("Done")

                            title  \
0                365 Data Science   
1                Alex The Analyst   
2          Data Science Analytics   
3    Learn data analytics with us   
4            MYANMAR DATA SCIENCE   
..                            ...   
253                 Andreas Kretz   
254       The Data Science Mentor   
255          EEHolmes-DataScience   
256                        Codeup   
258      SHARE STUDY แชร์สตั๊ดดี้   

                                           description  subscriber_count  \
0    At 365 Data Science we make #DataScience acces...            315000   
1    My name is Alex Freberg and on this channel I ...            784000   
2                                                                  12700   
3    In this channel our main aim is to teach diffe...              4410   
4    Providing free data science education in Myanm...             12100   
..                                                 ...               ...   
253  Hi my name is Andr